Intelligent Airline Customer Service System
Agentic AI Approach

In [20]:
!pip install transformers torch pandas matplotlib seaborn


In [21]:
import pandas as pd

df = pd.read_csv("/content/CS.csv")
df.head()


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [22]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


FILTER CUSTOMER QUERIES ONLY

In [23]:
customer_df = df[df["in_response_to_tweet_id"].isna()]
customer_df = customer_df[["text"]].dropna()
customer_df.head()


,text
6,@sprintcare is the worst customer service
12,@115714 y’all lie about your “great” connectio...
14,"@115714 whenever I contact customer support, t..."
23,actually that's a broken link you sent me and ...
25,"Yo @Ask_Spectrum, your customer service reps a..."


TEXT PREPROCESSING (ENCODING STEP)

In [24]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.strip()

customer_df["clean_text"] = customer_df["text"].apply(clean_text)


DEFINE AGENT INTENTS (AIRLINE SUPPORT)

In [25]:
AIRLINE_INTENTS = [
    "flight_delay",
    "refund_request",
    "baggage_issue",
    "booking_issue",
    "complaint",
    "general_inquiry"
]


AGENT 1: CUSTOMER INTAKE AGENT

In [26]:
def intake_agent(query):
    return query


AGENT 2: INTENT CLASSIFICATION AGENT (AI)

In [27]:
from transformers import pipeline

intent_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

def intent_agent(text):
    result = intent_classifier(text, AIRLINE_INTENTS)
    return result["labels"][0], result["scores"][0]


Device set to use cpu


In [28]:
sample_query = customer_df["clean_text"].iloc[0]
intent_agent(sample_query)


('complaint', 0.6734173893928528)

AGENT 3: AUTOMATED RESOLUTION AGENT

In [29]:
def resolution_agent(intent):
    responses = {
        "flight_delay": "We apologize for the delay. Please check your flight status on the airline app.",
        "refund_request": "Refund requests can be submitted via the airline refund portal.",
        "baggage_issue": "Please report baggage issues at the airport help desk or online.",
        "booking_issue": "Booking issues can be resolved through the Manage Booking section.",
        "general_inquiry": "Thank you for contacting us. How can we assist further?"
    }
    return responses.get(intent, None)


AGENT 4: ESCALATION AGENT (HUMAN HANDOVER)

In [30]:
def escalation_agent(query, intent, confidence):
    return f"""
    Escalation Required
    -------------------
    Reason: {intent}
    Confidence: {confidence:.2f}
    Customer Query: {query}
    Action: Forwarded to Human Airline Support Agent
    """


FULL AGENTIC ORCHESTRATION PIPELINE

In [31]:
def airline_support_system(user_query):
    # Agent 1
    query = intake_agent(user_query)

    # Agent 2
    intent, confidence = intent_agent(query)

    # Agent 3
    response = resolution_agent(intent)

    # Agent 4 (conditional escalation)
    if response and confidence > 0.6:
        return f"AI Response: {response}\nDetected Intent: {intent}"
    else:
        return escalation_agent(query, intent, confidence)


RUN COMPLETE WORKFLOW

In [32]:
test_query = customer_df["clean_text"].iloc[10]
print(airline_support_system(test_query))



    Escalation Required
    -------------------
    Reason: complaint
    Confidence: 0.56
    Customer Query: gtall of verizon is downlt
when can we expect a fix
    Action: Forwarded to Human Airline Support Agent
    


APPLY SYSTEM TO DATASET (SIMULATION)

In [34]:
customer_df = customer_df[customer_df["clean_text"] != ""]
sample_df = customer_df.sample(500, random_state=42)

sample_df["system_output"] = sample_df["clean_text"].apply(
    lambda x: airline_support_system(x)
)

sample_df.head()

,text,clean_text,system_output
1659182,@CoxHelp are there outages in Omaha? Been havi...,are there outages in omaha been having really ...,\n Escalation Required\n ---------------...
997283,Turns out @marksandspencer do have only one st...,turns out do have only one style of maternity...,\n Escalation Required\n ---------------...
1830820,@Ask_Spectrum My internet is still not working.,my internet is still not working,\n Escalation Required\n ---------------...
1336276,All these @115858 updates and my phone autocor...,all these updates and my phone autocorrecting...,\n Escalation Required\n ---------------...
1028500,So @Ask_Spectrum are you ever going to fix NBC...,so are you ever going to fix nbcsn on roku or...,\n Escalation Required\n ---------------...


SIMPLE ANALYSIS (SUPPORTS REPORT)

In [38]:
intent_counts = sample_df["system_output"].str.contains("Escalation").value_counts()
intent_counts

,count
system_output,
True,499
False,1
